# 🚀 Assistente de Pesquisa Inteligente com RAG e Agents

## Pedro Nunes Guth - Módulo 10: Projeto Final 1

Eaí pessoal! 🎉

Chegamos no **Módulo 10** e tá na hora de botar a mão na massa! Vamos criar um projeto que vai juntar **TUDO** que aprendemos até agora. 

Imagina ter um assistente que não só conversa contigo, mas também:
- 📚 Pesquisa em documentos (RAG)
- 🌐 Busca informações na internet (Agents)
- 🧠 Lembra do que vocês já conversaram (Memory)
- 🎯 Responde de forma estruturada (Output Parsers)

É como ter um estagiário super inteligente que nunca esquece de nada e ainda pesquisa tudo pra você!

**Dica do Pedro**: Este projeto vai servir de base para o Projeto Final 2 e o deploy no Streamlit. Então capricha!

## 🎯 O que vamos construir?

Nosso **Assistente de Pesquisa Inteligente** vai ter:

### Funcionalidades:
1. **RAG System**: Consulta documentos locais
2. **Web Agent**: Pesquisa informações online
3. **Memory**: Mantém contexto da conversa
4. **Router**: Decide qual ferramenta usar
5. **Output Parsing**: Formata respostas estruturadas

### Arquitetura:
```mermaid
graph TD
    A[Usuário] --> B[Router Agent]
    B --> C{Tipo de Pergunta?}
    C -->|Documentos| D[RAG System]
    C -->|Web Search| E[Web Agent]
    C -->|Conversa| F[Chat Agent]
    D --> G[Memory + Response]
    E --> G
    F --> G
    G --> H[Output Parser]
    H --> I[Resposta Estruturada]
```

Tá, mas por que essa arquitetura? Porque na vida real, um assistente precisa saber **quando** usar **qual** ferramenta. É como um canivete suíço inteligente!

In [ ]:
# Setup inicial - Vamos instalar todas as dependências necessárias
!pip install -q langchain langchain-google-genai langchain-community
!pip install -q faiss-cpu pypdf sentence-transformers
!pip install -q duckduckgo-search wikipedia-api requests beautifulsoup4
!pip install -q python-dotenv

In [ ]:
# Imports necessários - Tudo que aprendemos nos módulos anteriores!
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# LangChain Core
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser, StructuredOutputParser
from langchain.schema.output_parser import OutputParserException

# Chat Model
from langchain_google_genai import ChatGoogleGenerativeAI

# RAG Components
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# Agents and Tools
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Utils
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
import json
from datetime import datetime

print("📦 Imports carregados! Bora pro código!")

In [ ]:
# Configuração da API do Google Gemini
# Lembre-se de configurar sua GOOGLE_API_KEY
load_dotenv()

# Se você não tem .env, descomente e coloque sua chave aqui:
# os.environ["GOOGLE_API_KEY"] = "sua_chave_aqui"

# Inicializando nosso modelo principal - o Gemini 2.0 Flash
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.3,
    max_tokens=1000
)

print("🤖 Modelo Gemini carregado e pronto para action!")

# Testando rapidinho
response = llm.invoke("Diga apenas: Funcionando!")
print(f"✅ Teste: {response.content}")

## 📚 Componente 1: Sistema RAG

Lembra do **Módulo 8**? Vamos usar tudo que aprendemos sobre RAG! 

O RAG (Retrieval-Augmented Generation) é como ter uma biblioteca pessoal super organizada. Você pergunta algo, ele:
1. 🔍 Procura nos documentos relevantes
2. 📖 Pega os trechos mais importantes  
3. 🧠 Gera uma resposta baseada no conteúdo

### Matemática por trás:

O processo de similaridade usa **cosine similarity**:

$$similarity(A, B) = \frac{A \cdot B}{||A|| \times ||B||}$$

Onde:
- $A$ é o embedding da pergunta
- $B$ é o embedding do documento
- Resultado varia de -1 a 1 (mais próximo de 1 = mais similar)

**Dica do Pedro**: Pense no embedding como as "coordenadas" do significado de um texto no espaço multidimensional!

In [ ]:
class RAGSystem:
    """Sistema RAG completo - Do Módulo 8 turbinado!"""
    
    def __init__(self, llm):
        self.llm = llm
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        self.vectorstore = None
        self.qa_chain = None
        
    def add_documents_from_text(self, texts: List[str], metadatas: List[Dict] = None):
        """Adiciona documentos de texto diretamente"""
        try:
            # Processa os textos
            all_chunks = []
            all_metadatas = []
            
            for i, text in enumerate(texts):
                chunks = self.text_splitter.split_text(text)
                all_chunks.extend(chunks)
                
                # Metadados para cada chunk
                base_metadata = metadatas[i] if metadatas else {}
                chunk_metadatas = [{**base_metadata, 'chunk_id': j} 
                                 for j in range(len(chunks))]
                all_metadatas.extend(chunk_metadatas)
            
            # Cria ou atualiza o vectorstore
            if self.vectorstore is None:
                self.vectorstore = FAISS.from_texts(
                    all_chunks, 
                    self.embeddings,
                    metadatas=all_metadatas
                )
            else:
                # Adiciona ao vectorstore existente
                new_vectorstore = FAISS.from_texts(
                    all_chunks, 
                    self.embeddings,
                    metadatas=all_metadatas
                )
                self.vectorstore.merge_from(new_vectorstore)
            
            # Cria a chain de QA
            self._setup_qa_chain()
            
            return f"✅ {len(all_chunks)} chunks adicionados com sucesso!"
            
        except Exception as e:
            return f"❌ Erro ao processar documentos: {str(e)}"
    
    def _setup_qa_chain(self):
        """Configura a chain de Question Answering"""
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
    
    def query(self, question: str) -> Dict[str, Any]:
        """Faz uma consulta no sistema RAG"""
        if self.qa_chain is None:
            return {
                "answer": "❌ Nenhum documento foi carregado ainda.",
                "sources": []
            }
        
        try:
            result = self.qa_chain({"query": question})
            
            return {
                "answer": result["result"],
                "sources": [doc.metadata for doc in result["source_documents"]],
                "source_texts": [doc.page_content[:200] + "..." 
                               for doc in result["source_documents"]]
            }
            
        except Exception as e:
            return {
                "answer": f"❌ Erro na consulta: {str(e)}",
                "sources": []
            }

# Instanciando nosso sistema RAG
rag_system = RAGSystem(llm)
print("📚 Sistema RAG inicializado! Vamos adicionar alguns documentos...")

In [ ]:
# Vamos adicionar alguns documentos de exemplo sobre IA e LangChain
sample_documents = [
    """
    LangChain é um framework para desenvolvimento de aplicações com Large Language Models (LLMs). 
    Ele fornece abstrações e ferramentas para:
    - Prompt Templates: Para estruturar entradas para LLMs
    - Chains: Para sequenciar operações
    - Agents: Para tomada de decisões dinâmicas
    - Memory: Para manter contexto entre interações
    - Document Loaders: Para processar diferentes tipos de dados
    - Vector Stores: Para busca semântica
    
    A versão v0.2 do LangChain introduziu melhorias significativas na API e performance.
    """,
    
    """
    Retrieval-Augmented Generation (RAG) é uma técnica que combina recuperação de informações 
    com geração de texto. O processo funciona assim:
    1. Documentos são divididos em chunks menores
    2. Chunks são convertidos em embeddings vetoriais
    3. Para uma pergunta, busca-se chunks similares
    4. Chunks relevantes são fornecidos como contexto para o LLM
    5. LLM gera resposta baseada no contexto recuperado
    
    RAG resolve o problema de conhecimento desatualizado dos LLMs e permite 
    consultas em bases de conhecimento específicas.
    """,
    
    """
    Agents em LangChain são sistemas que podem tomar decisões sobre quais 
    ferramentas usar para responder perguntas. Tipos principais:
    
    - Zero-shot ReAct: Usa reasoning e acting em ciclos
    - Conversational ReAct: Mantém memória de conversas
    - Plan-and-execute: Planeja e executa etapas sequencialmente
    
    Tools são funções que agents podem chamar, como:
    - Web search (DuckDuckGo, Google)
    - Calculadora
    - APIs externas
    - Sistemas de arquivos
    
    O processo ReAct segue: Thought -> Action -> Observation -> Thought...
    """
]

# Metadados para os documentos
metadatas = [
    {"source": "langchain_docs", "topic": "framework", "date": "2024-01-15"},
    {"source": "rag_guide", "topic": "rag", "date": "2024-01-10"},
    {"source": "agents_manual", "topic": "agents", "date": "2024-01-20"}
]

# Adicionando documentos ao sistema RAG
result = rag_system.add_documents_from_text(sample_documents, metadatas)
print(result)

# Testando o sistema
print("\n🧪 Testando consulta RAG...")
test_query = "O que é RAG e como funciona?"
rag_result = rag_system.query(test_query)

print(f"\n❓ Pergunta: {test_query}")
print(f"\n📋 Resposta: {rag_result['answer']}")
print(f"\n📚 Sources: {len(rag_result['sources'])} documentos consultados")

## 🌐 Componente 2: Web Agent

Agora vamos implementar nosso **Web Agent**! Lembra do **Módulo 9** sobre Agents?

Um Web Agent é como ter um assistente de pesquisa que:
- 🔍 Sabe onde buscar informações
- 🧠 Raciocina sobre o que encontrou
- 🎯 Filtra o que é relevante
- 📝 Resume tudo numa resposta útil

### O Ciclo ReAct:
O agent segue o padrão **ReAct** (Reasoning + Acting):

1. **Thought**: "Preciso buscar informações sobre X"
2. **Action**: Usa ferramenta de busca
3. **Observation**: Analisa os resultados
4. **Thought**: "Encontrei Y, mas preciso de mais detalhes sobre Z"
5. **Action**: Nova busca mais específica
6. **Final Answer**: Resposta consolidada

É como o processo mental que você faz quando pesquisa algo no Google!

In [ ]:
class WebAgent:
    """Agent para pesquisas na web - Módulo 9 na prática!"""
    
    def __init__(self, llm):
        self.llm = llm
        self.tools = self._setup_tools()
        self.agent = self._setup_agent()
    
    def _setup_tools(self):
        """Configura as ferramentas disponíveis para o agent"""
        tools = []
        
        # DuckDuckGo Search - Busca geral na web
        try:
            search = DuckDuckGoSearchRun()
            search_tool = Tool(
                name="Web Search",
                func=search.run,
                description="Útil para buscar informações atuais na internet. Use para perguntas sobre eventos recentes, notícias, ou informações que podem ter mudado."
            )
            tools.append(search_tool)
        except Exception as e:
            print(f"⚠️ Erro ao configurar DuckDuckGo: {e}")
        
        # Wikipedia - Para informações enciclopédicas
        try:
            wikipedia = WikipediaQueryRun(
                api_wrapper=WikipediaAPIWrapper()
            )
            wiki_tool = Tool(
                name="Wikipedia",
                func=wikipedia.run,
                description="Útil para buscar informações enciclopédicas, definições, história, biografias e conhecimento geral bem estabelecido."
            )
            tools.append(wiki_tool)
        except Exception as e:
            print(f"⚠️ Erro ao configurar Wikipedia: {e}")
        
        return tools
    
    def _setup_agent(self):
        """Configura o agent ReAct"""
        if not self.tools:
            print("❌ Nenhuma ferramenta disponível para o agent")
            return None
            
        try:
            agent = initialize_agent(
                tools=self.tools,
                llm=self.llm,
                agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                verbose=False,
                max_iterations=3,
                early_stopping_method="generate"
            )
            return agent
        except Exception as e:
            print(f"❌ Erro ao criar agent: {e}")
            return None
    
    def search(self, query: str) -> Dict[str, Any]:
        """Executa uma pesquisa usando o agent"""
        if self.agent is None:
            return {
                "answer": "❌ Agent não disponível. Verifique as ferramentas.",
                "tools_used": [],
                "success": False
            }
        
        try:
            # Executa o agent
            result = self.agent.run(query)
            
            return {
                "answer": result,
                "tools_used": [tool.name for tool in self.tools],
                "success": True
            }
            
        except Exception as e:
            return {
                "answer": f"❌ Erro na pesquisa: {str(e)}",
                "tools_used": [],
                "success": False
            }
    
    def get_available_tools(self):
        """Retorna lista de ferramentas disponíveis"""
        return [{
            "name": tool.name,
            "description": tool.description
        } for tool in self.tools]

# Instanciando nosso Web Agent
web_agent = WebAgent(llm)
print("🌐 Web Agent inicializado!")
print(f"🛠️ Ferramentas disponíveis: {len(web_agent.tools)}")

# Mostrando as ferramentas
for tool_info in web_agent.get_available_tools():
    print(f"  - {tool_info['name']}: {tool_info['description'][:50]}...")

In [ ]:
# Testando o Web Agent
print("🧪 Testando Web Agent...\n")

# Teste 1: Busca simples
test_query_1 = "Quais são as principais notícias sobre inteligência artificial hoje?"
print(f"❓ Teste 1: {test_query_1}")

result_1 = web_agent.search(test_query_1)
print(f"✅ Sucesso: {result_1['success']}")
print(f"📝 Resposta: {result_1['answer'][:300]}...")
print(f"🛠️ Tools usadas: {result_1['tools_used']}")

print("\n" + "="*50 + "\n")

# Teste 2: Busca enciclopédica
test_query_2 = "O que é machine learning?"
print(f"❓ Teste 2: {test_query_2}")

result_2 = web_agent.search(test_query_2)
print(f"✅ Sucesso: {result_2['success']}")
print(f"📝 Resposta: {result_2['answer'][:300]}...")
print(f"🛠️ Tools usadas: {result_2['tools_used']}")

## 🧠 Componente 3: Sistema de Memória

Lembra do **Módulo 5** sobre Memory Systems? Agora vamos implementar um sistema de memória robusto!

Por que memória é importante?
- 📝 Mantém contexto da conversa
- 🔄 Permite referências a mensagens anteriores
- 🎯 Melhora a relevância das respostas
- 👤 Personaliza a experiência do usuário

### Tipos de Memória:
1. **Buffer Memory**: Últimas N mensagens
2. **Summary Memory**: Resume conversas longas
3. **Entity Memory**: Lembra de entidades específicas

Vamos usar **ConversationBufferWindowMemory** - é como ter uma memória de curto prazo que lembra das últimas interações!

In [ ]:
class MemoryManager:
    """Gerenciador de memória - Módulo 5 turbinado!"""
    
    def __init__(self, window_size: int = 10):
        self.window_size = window_size
        self.memory = ConversationBufferWindowMemory(
            k=window_size,
            return_messages=True,
            memory_key="chat_history"
        )
        self.conversation_stats = {
            "total_messages": 0,
            "rag_queries": 0,
            "web_searches": 0,
            "start_time": datetime.now()
        }
    
    def add_interaction(self, human_message: str, ai_message: str, interaction_type: str = "chat"):
        """Adiciona uma interação à memória"""
        # Adiciona à memória do LangChain
        self.memory.chat_memory.add_user_message(human_message)
        self.memory.chat_memory.add_ai_message(ai_message)
        
        # Atualiza estatísticas
        self.conversation_stats["total_messages"] += 1
        if interaction_type == "rag":
            self.conversation_stats["rag_queries"] += 1
        elif interaction_type == "web":
            self.conversation_stats["web_searches"] += 1
    
    def get_context(self) -> str:
        """Retorna o contexto atual da conversa"""
        messages = self.memory.chat_memory.messages
        if not messages:
            return "Nenhuma conversa anterior."
        
        context_parts = []
        for msg in messages[-6:]:  # Últimas 3 interações (6 mensagens)
            if isinstance(msg, HumanMessage):
                context_parts.append(f"Humano: {msg.content}")
            elif isinstance(msg, AIMessage):
                context_parts.append(f"Assistente: {msg.content}")
        
        return "\n".join(context_parts)
    
    def get_stats(self) -> Dict[str, Any]:
        """Retorna estatísticas da conversa"""
        duration = datetime.now() - self.conversation_stats["start_time"]
        
        return {
            **self.conversation_stats,
            "duration_minutes": duration.total_seconds() / 60,
            "messages_in_memory": len(self.memory.chat_memory.messages),
            "memory_window_size": self.window_size
        }
    
    def clear_memory(self):
        """Limpa a memória e reinicia estatísticas"""
        self.memory.clear()
        self.conversation_stats = {
            "total_messages": 0,
            "rag_queries": 0,
            "web_searches": 0,
            "start_time": datetime.now()
        }
    
    def export_conversation(self) -> List[Dict[str, str]]:
        """Exporta a conversa atual"""
        messages = self.memory.chat_memory.messages
        conversation = []
        
        for msg in messages:
            if isinstance(msg, HumanMessage):
                conversation.append({
                    "type": "human",
                    "content": msg.content,
                    "timestamp": datetime.now().isoformat()
                })
            elif isinstance(msg, AIMessage):
                conversation.append({
                    "type": "ai",
                    "content": msg.content,
                    "timestamp": datetime.now().isoformat()
                })
        
        return conversation

# Instanciando o gerenciador de memória
memory_manager = MemoryManager(window_size=8)
print("🧠 Sistema de memória inicializado!")
print(f"📊 Window size: {memory_manager.window_size} interações")

# Teste rápido
memory_manager.add_interaction(
    "Olá! Como você funciona?",
    "Olá! Sou um assistente inteligente que combina RAG, web search e memória!",
    "chat"
)

print("\n📈 Stats iniciais:")
stats = memory_manager.get_stats()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

## 🎯 Componente 4: Router Inteligente

Agora a parte mais legal! O **Router** é o cérebro que decide qual ferramenta usar. É como um mordomo super inteligente que sabe exatamente o que você precisa!

### Lógica de Decisão:
O router analisa a pergunta e decide:

- 📚 **RAG**: Se a pergunta é sobre os documentos carregados
- 🌐 **Web Search**: Se precisa de informações atuais/externas
- 💬 **Chat**: Se é conversa casual ou já tem contexto suficiente

### Algoritmo de Classificação:
Usamos um **LLM como classificador** com prompt engenheirado:

$$P(categoria|pergunta) = \frac{e^{score_{categoria}}}{\sum_{i} e^{score_i}}$$

É como ter um "sexto sentido" para saber qual ferramenta é perfeita para cada situação!

In [ ]:
# Primeiro, vamos criar um Output Parser para estruturar as respostas
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class RouterDecision(BaseModel):
    """Estrutura da decisão do router"""
    route: str = Field(description="A rota escolhida: 'rag', 'web', ou 'chat'")
    confidence: float = Field(description="Confiança na decisão (0.0 a 1.0)")
    reasoning: str = Field(description="Justificativa para a escolha")

class AssistantResponse(BaseModel):
    """Estrutura da resposta final do assistente"""
    answer: str = Field(description="Resposta principal")
    source_type: str = Field(description="Tipo da fonte: 'rag', 'web', 'chat', 'memory'")
    confidence: float = Field(description="Confiança na resposta")
    sources: List[str] = Field(description="Fontes consultadas (se aplicável)")
    context_used: bool = Field(description="Se usou contexto da conversa")

# Parsers
router_parser = PydanticOutputParser(pydantic_object=RouterDecision)
response_parser = PydanticOutputParser(pydantic_object=AssistantResponse)

print("📋 Output Parsers criados! Estrutura definida.")

In [ ]:
class IntelligentRouter:
    """Router inteligente - O cérebro do nosso assistente!"""
    
    def __init__(self, llm, rag_system, web_agent, memory_manager):
        self.llm = llm
        self.rag_system = rag_system
        self.web_agent = web_agent
        self.memory_manager = memory_manager
        
        # Template para decisão de rota
        self.router_template = PromptTemplate(
            input_variables=["question", "context", "available_docs"],
            template="""Você é um router inteligente que decide qual ferramenta usar para responder perguntas.

Ferramentas disponíveis:
- 'rag': Para consultar documentos locais sobre LangChain, RAG, Agents, etc.
- 'web': Para buscar informações atuais na internet ou tópicos não cobertos nos docs
- 'chat': Para conversas casuais ou quando já há contexto suficiente

Contexto da conversa:
{context}

Documentos disponíveis no RAG: {available_docs}

Pergunta: {question}

Analise a pergunta e decida qual ferramenta usar. Considere:
- Se a pergunta é sobre LangChain/RAG/Agents → use 'rag'
- Se precisa de informações atuais/externas → use 'web'  
- Se é conversa casual ou continua o contexto → use 'chat'

{format_instructions}
""",
            partial_variables={"format_instructions": router_parser.get_format_instructions()}
        )
    
    def decide_route(self, question: str) -> RouterDecision:
        """Decide qual rota usar para a pergunta"""
        try:
            # Pega contexto da conversa
            context = self.memory_manager.get_context()
            
            # Informações sobre documentos disponíveis
            available_docs = "LangChain framework, RAG implementation, Agents and Tools"
            
            # Forma o prompt
            prompt = self.router_template.format(
                question=question,
                context=context,
                available_docs=available_docs
            )
            
            # Gera decisão
            response = self.llm.invoke(prompt)
            decision = router_parser.parse(response.content)
            
            return decision
            
        except Exception as e:
            # Fallback simples baseado em palavras-chave
            question_lower = question.lower()
            
            if any(word in question_lower for word in ['langchain', 'rag', 'agent', 'embedding', 'vector']):
                route = 'rag'
                reasoning = "Detectadas palavras-chave relacionadas aos documentos"
            elif any(word in question_lower for word in ['atual', 'hoje', 'notícia', 'recente', 'agora']):
                route = 'web'
                reasoning = "Detectadas palavras-chave indicando necessidade de informações atuais"
            else:
                route = 'chat'
                reasoning = "Fallback para chat conversacional"
            
            return RouterDecision(
                route=route,
                confidence=0.6,
                reasoning=f"Fallback usado devido ao erro: {str(e)}. {reasoning}"
            )
    
    def process_question(self, question: str) -> AssistantResponse:
        """Processa uma pergunta completa usando a rota apropriada"""
        # Decide a rota
        decision = self.decide_route(question)
        
        print(f"🎯 Rota escolhida: {decision.route} (confiança: {decision.confidence:.2f})")
        print(f"🤔 Reasoning: {decision.reasoning}")
        
        # Executa baseado na decisão
        if decision.route == 'rag':
            return self._handle_rag(question, decision)
        elif decision.route == 'web':
            return self._handle_web(question, decision)
        else:
            return self._handle_chat(question, decision)
    
    def _handle_rag(self, question: str, decision: RouterDecision) -> AssistantResponse:
        """Processa pergunta usando RAG"""
        rag_result = self.rag_system.query(question)
        
        # Adiciona à memória
        self.memory_manager.add_interaction(question, rag_result['answer'], 'rag')
        
        return AssistantResponse(
            answer=rag_result['answer'],
            source_type='rag',
            confidence=decision.confidence,
            sources=[f"Documento: {src.get('source', 'unknown')}" for src in rag_result.get('sources', [])],
            context_used=True
        )
    
    def _handle_web(self, question: str, decision: RouterDecision) -> AssistantResponse:
        """Processa pergunta usando Web Search"""
        web_result = self.web_agent.search(question)
        
        # Adiciona à memória
        self.memory_manager.add_interaction(question, web_result['answer'], 'web')
        
        return AssistantResponse(
            answer=web_result['answer'],
            source_type='web',
            confidence=decision.confidence if web_result['success'] else 0.3,
            sources=web_result.get('tools_used', []),
            context_used=False
        )
    
    def _handle_chat(self, question: str, decision: RouterDecision) -> AssistantResponse:
        """Processa pergunta usando chat simples com contexto"""
        context = self.memory_manager.get_context()
        
        chat_prompt = f"""
Contexto da conversa:
{context}

Pergunta atual: {question}

Responda de forma natural e útil, considerando o contexto da conversa.
"""
        
        response = self.llm.invoke(chat_prompt)
        answer = response.content
        
        # Adiciona à memória
        self.memory_manager.add_interaction(question, answer, 'chat')
        
        return AssistantResponse(
            answer=answer,
            source_type='chat',
            confidence=decision.confidence,
            sources=['Conversa contextual'],
            context_used=bool(context and context != "Nenhuma conversa anterior.")
        )

# Instanciando nosso Router Inteligente
router = IntelligentRouter(llm, rag_system, web_agent, memory_manager)
print("🎯 Router Inteligente criado! Agora temos o cérebro do assistente!")

## 🧪 Teste Completo do Sistema

Agora vamos testar nosso **Assistente de Pesquisa Inteligente** completo!

Vamos simular uma conversa real para ver como ele:
- 🎯 Escolhe a ferramenta certa
- 🧠 Usa a memória 
- 📊 Mantém estatísticas
- 🔄 Alterna entre diferentes fontes

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-10_img_01.png)

In [ ]:
def test_assistant(questions: List[str]):
    """Testa o assistente com uma série de perguntas"""
    print("🚀 Iniciando teste completo do Assistente de Pesquisa Inteligente!\n")
    print("="*70)
    
    for i, question in enumerate(questions, 1):
        print(f"\n🎯 PERGUNTA {i}: {question}")
        print("-" * 50)
        
        # Processa a pergunta
        response = router.process_question(question)
        
        # Mostra a resposta
        print(f"\n📋 RESPOSTA ({response.source_type.upper()}):")
        print(response.answer)
        
        print(f"\n📊 METADADOS:")
        print(f"  • Confiança: {response.confidence:.2f}")
        print(f"  • Contexto usado: {response.context_used}")
        print(f"  • Fontes: {', '.join(response.sources)}")
        
        print("\n" + "="*70)
    
    # Estatísticas finais
    print("\n📈 ESTATÍSTICAS FINAIS:")
    stats = memory_manager.get_stats()
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"  • {key}: {value:.2f}")
        elif key != 'start_time':
            print(f"  • {key}: {value}")

# Perguntas de teste que cobrem diferentes cenários
test_questions = [
    "Olá! Como você funciona?",  # Chat inicial
    "O que é RAG e como implementar?",  # RAG query
    "Quais são as últimas notícias sobre IA?",  # Web search
    "Você mencionou RAG antes, pode explicar melhor sobre embeddings?",  # RAG com contexto
    "E como posso usar agents no meu projeto?",  # RAG sobre agents
]

# Executa o teste
test_assistant(test_questions)

## 📊 Visualização da Performance

Vamos criar alguns gráficos para visualizar como nosso assistente está performando!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dados das estatísticas
stats = memory_manager.get_stats()

# Gráfico 1: Distribuição de tipos de consulta
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico de pizza - Tipos de consulta
query_types = ['RAG Queries', 'Web Searches', 'Chat Messages']
query_counts = [
    stats['rag_queries'], 
    stats['web_searches'], 
    stats['total_messages'] - stats['rag_queries'] - stats['web_searches']
]

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
ax1.pie(query_counts, labels=query_types, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribuição de Tipos de Consulta', fontsize=14, fontweight='bold')

# Gráfico 2: Timeline simulada de confiança
interactions = list(range(1, stats['total_messages'] + 1))
# Simulando scores de confiança (na prática, você salvaria estes valores)
confidence_scores = np.random.uniform(0.7, 0.95, len(interactions))

ax2.plot(interactions, confidence_scores, 'o-', color='#45B7D1', linewidth=2, markersize=6)
ax2.set_title('Evolução da Confiança nas Respostas', fontsize=14, fontweight='bold')
ax2.set_xlabel('Número da Interação')
ax2.set_ylabel('Score de Confiança')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 1)

# Gráfico 3: Comparação de ferramentas disponíveis
tools = ['RAG System', 'Web Agent', 'Memory', 'Router']
effectiveness = [0.9, 0.85, 0.95, 0.88]  # Valores simulados

bars = ax3.bar(tools, effectiveness, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
ax3.set_title('Efetividade dos Componentes', fontsize=14, fontweight='bold')
ax3.set_ylabel('Score de Efetividade')
ax3.set_ylim(0, 1)

# Adiciona valores nas barras
for bar, value in zip(bars, effectiveness):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.2f}', ha='center', fontweight='bold')

# Gráfico 4: Métricas de memória
memory_metrics = ['Mensagens\nTotais', 'Mensagens\nem Memória', 'Window\nSize']
memory_values = [stats['total_messages'], stats['messages_in_memory'], stats['memory_window_size']]

bars2 = ax4.bar(memory_metrics, memory_values, color=['#FECA57', '#FF9FF3', '#54A0FF'])
ax4.set_title('Métricas do Sistema de Memória', fontsize=14, fontweight='bold')
ax4.set_ylabel('Quantidade')

# Adiciona valores nas barras
for bar, value in zip(bars2, memory_values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{int(value)}', ha='center', fontweight='bold')

plt.tight_layout()
plt.suptitle('📊 Dashboard do Assistente de Pesquisa Inteligente', 
             fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("\n📊 Dashboard gerado! Estes gráficos mostram o desempenho do nosso sistema.")
print("💡 No Módulo 12, vamos colocar isso tudo numa interface Streamlit linda!")

## 🏗️ Arquitetura Final do Sistema

Vamos visualizar nossa arquitetura completa!

```mermaid
graph TB
    subgraph "Interface"
        UI[Usuário]
    end
    
    subgraph "Core System"
        R[Router Inteligente]
        M[Memory Manager]
    end
    
    subgraph "Ferramentas"
        RAG[RAG System]
        WEB[Web Agent]
        CHAT[Chat Engine]
    end
    
    subgraph "Dados"
        DOCS[Documentos]
        VECTOR[Vector Store]
        INTERNET[Internet]
    end
    
    subgraph "AI Models"
        LLM[Gemini 2.0 Flash]
        EMB[Embeddings]
    end
    
    UI --> R
    R --> M
    R --> RAG
    R --> WEB
    R --> CHAT
    
    RAG --> VECTOR
    RAG --> EMB
    VECTOR --> DOCS
    
    WEB --> INTERNET
    
    RAG --> LLM
    WEB --> LLM
    CHAT --> LLM
    R --> LLM
    
    M -.-> RAG
    M -.-> WEB
    M -.-> CHAT
```

### Fluxo de Dados:
1. **Input** → Router analisa a pergunta
2. **Decisão** → Router escolhe ferramenta apropriada
3. **Execução** → Ferramenta processa com LLM/dados
4. **Memória** → Resultado é armazenado para contexto
5. **Output** → Resposta estruturada é retornada

In [ ]:
# Vamos criar uma função de conveniência para usar nosso assistente
class SmartResearchAssistant:
    """Assistente de Pesquisa Inteligente - Interface principal"""
    
    def __init__(self):
        self.router = router
        self.memory = memory_manager
        self.conversation_id = f"conv_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
    def ask(self, question: str, verbose: bool = True) -> Dict[str, Any]:
        """Interface principal para fazer perguntas"""
        if verbose:
            print(f"\n❓ {question}")
            print("-" * 50)
        
        # Processa a pergunta
        response = self.router.process_question(question)
        
        if verbose:
            print(f"\n🤖 {response.answer}")
            print(f"\n📊 Fonte: {response.source_type} | Confiança: {response.confidence:.2f}")
            if response.sources:
                print(f"📚 Fontes: {', '.join(response.sources[:2])}")
        
        return {
            "question": question,
            "answer": response.answer,
            "source_type": response.source_type,
            "confidence": response.confidence,
            "sources": response.sources,
            "context_used": response.context_used,
            "conversation_id": self.conversation_id
        }
    
    def get_stats(self):
        """Retorna estatísticas da conversa"""
        return self.memory.get_stats()
    
    def export_conversation(self):
        """Exporta a conversa atual"""
        return self.memory.export_conversation()
    
    def reset(self):
        """Reinicia a conversa"""
        self.memory.clear_memory()
        self.conversation_id = f"conv_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        print("🔄 Conversa reiniciada!")

# Instancia nosso assistente final
assistant = SmartResearchAssistant()
print("🎉 Assistente de Pesquisa Inteligente está pronto para uso!")
print(f"🆔 ID da conversa: {assistant.conversation_id}")
print("\n💡 Use assistant.ask('sua pergunta') para interagir!")

In [ ]:
# Demonstração final do assistente
print("🚀 DEMONSTRAÇÃO FINAL - Assistente de Pesquisa Inteligente\n")

# Exemplo 1: Pergunta sobre RAG (deve usar RAG system)
result1 = assistant.ask("Como funciona o processo de embedding no RAG?")

# Exemplo 2: Pergunta atual (deve usar web search)
result2 = assistant.ask("Quais são as tendências de IA em 2024?")

# Exemplo 3: Continuação da conversa (deve usar contexto)
result3 = assistant.ask("Pode me dar mais detalhes sobre a primeira pergunta que fiz?")

print("\n" + "="*60)
print("📈 RESUMO DA DEMONSTRAÇÃO:")
stats = assistant.get_stats()
print(f"  • Total de mensagens: {stats['total_messages']}")
print(f"  • Consultas RAG: {stats['rag_queries']}")
print(f"  • Buscas web: {stats['web_searches']}")
print(f"  • Duração: {stats['duration_minutes']:.1f} minutos")
print(f"  • Mensagens em memória: {stats['messages_in_memory']}")

## 🎯 Exercícios Práticos

Agora é sua vez de praticar! Vamos fazer alguns exercícios para fixar o aprendizado.

**Dica do Pedro**: Estes exercícios vão te preparar para o Projeto Final 2 e para trabalhar com sistemas mais complexos!

### 🚀 Exercício 1: Melhorando o Router

**Desafio**: Implemente uma versão melhorada do router que:
1. Considera o histórico de rotas usadas
2. Tem uma confiança mínima antes de usar web search
3. Pode combinar múltiplas fontes numa resposta

**Código inicial**:
```python
class AdvancedRouter(IntelligentRouter):
    def __init__(self, llm, rag_system, web_agent, memory_manager):
        super().__init__(llm, rag_system, web_agent, memory_manager)
        self.route_history = []
        self.min_confidence_web = 0.8
    
    def decide_route(self, question: str):
        # SEU CÓDIGO AQUI
        # Implemente a lógica melhorada
        pass
```

**Teste sua implementação** com diferentes tipos de pergunta!

In [ ]:
# Espaço para o Exercício 1
# Implemente seu AdvancedRouter aqui!

class AdvancedRouter(IntelligentRouter):
    def __init__(self, llm, rag_system, web_agent, memory_manager):
        super().__init__(llm, rag_system, web_agent, memory_manager)
        self.route_history = []
        self.min_confidence_web = 0.8
    
    def decide_route(self, question: str):
        # Sua implementação aqui!
        # Dicas:
        # - Use self.route_history para análise de padrões
        # - Considere self.min_confidence_web para web searches
        # - Pense em como combinar RAG + Web quando necessário
        
        # Placeholder - implemente sua versão!
        decision = super().decide_route(question)
        self.route_history.append(decision.route)
        return decision

print("✏️ Exercício 1: Implemente seu AdvancedRouter acima!")
print("💡 Dica: Pense em como um assistente real tomaria decisões mais inteligentes.")

### 🎨 Exercício 2: Sistema de Cache Inteligente

**Desafio**: Crie um sistema de cache que:
1. Armazena respostas de web searches para evitar buscas repetidas
2. Tem TTL (Time To Live) configurável
3. Usa similaridade semântica para encontrar cache hits

**Por que é importante?**
- 💰 Economiza API calls
- ⚡ Melhora performance
- 🌍 Reduz impacto ambiental

**Estrutura sugerida**:
```python
class SmartCache:
    def __init__(self, ttl_minutes=60, similarity_threshold=0.85):
        # SEU CÓDIGO AQUI
        pass
    
    def get(self, query: str):
        # Busca no cache usando similaridade
        pass
    
    def set(self, query: str, response: str):
        # Adiciona ao cache com timestamp
        pass
```

In [ ]:
# Espaço para o Exercício 2
# Implemente seu SmartCache aqui!

from datetime import datetime, timedelta
import pickle

class SmartCache:
    def __init__(self, ttl_minutes=60, similarity_threshold=0.85):
        self.ttl_minutes = ttl_minutes
        self.similarity_threshold = similarity_threshold
        self.cache = {}  # {query_hash: {response, timestamp, query_text}}
        
        # Você vai precisar de embeddings para comparar similaridade
        # Dica: Use o mesmo modelo de embeddings do RAG!
    
    def get(self, query: str):
        # Sua implementação aqui!
        # Passos sugeridos:
        # 1. Calcule embedding da query
        # 2. Compare com queries em cache
        # 3. Verifique TTL
        # 4. Retorne se similar + válido
        
        print(f"🔍 Buscando '{query}' no cache... (não implementado ainda)")
        return None
    
    def set(self, query: str, response: str):
        # Sua implementação aqui!
        # Passos sugeridos:
        # 1. Calcule hash/embedding da query
        # 2. Armazene com timestamp
        # 3. Limpe entradas antigas
        
        print(f"💾 Salvando resposta para '{query}' no cache... (não implementado ainda)")
    
    def cleanup_expired(self):
        # Remove entradas expiradas
        pass
    
    def get_stats(self):
        # Retorna estatísticas do cache
        return {
            "total_entries": len(self.cache),
            "ttl_minutes": self.ttl_minutes,
            "similarity_threshold": self.similarity_threshold
        }

# Teste básico
cache = SmartCache()
print("✏️ Exercício 2: Implemente seu SmartCache acima!")
print(f"📊 Stats do cache: {cache.get_stats()}")
print("💡 Dica: Use cosine similarity para comparar embeddings de queries!")

## 🎉 Resumo e Próximos Passos

**Liiindo!** 🚀 Chegamos ao fim do **Módulo 10** e olha só o que construímos:

### ✅ O que implementamos:
1. **Sistema RAG completo** - Consulta documentos com embeddings
2. **Web Agent inteligente** - Busca informações online com ReAct
3. **Sistema de Memória** - Mantém contexto da conversa
4. **Router Inteligente** - Decide qual ferramenta usar
5. **Output Parsing** - Estrutura respostas consistentes
6. **Interface unificada** - SmartResearchAssistant

### 🧠 Conceitos aplicados:
- **Módulo 2**: ChatModel com Gemini 2.0 Flash
- **Módulo 3**: PromptTemplates e OutputParsers 
- **Módulo 4**: Chains para processamento
- **Módulo 5**: Memory para contexto
- **Módulo 6-7**: Document Loading e Vector Stores
- **Módulo 8**: RAG Implementation
- **Módulo 9**: Agents e Tools

### 🔮 O que vem por aí:
- **Módulo 11**: Projeto Final 2 (ainda mais avançado!)
- **Módulo 12**: Deploy com Streamlit 
- **Módulo 13**: Migração para v1.0
- **Módulo 14-15**: LangGraph e LangSmith

**Dica do Pedro**: Este projeto é sua base! Salve o código, experimente melhorias e prepare-se para o próximo nível! 🎯

In [ ]:
# Função final para salvar o projeto
def save_project_summary():
    """Salva um resumo do projeto desenvolvido"""
    
    summary = {
        "project_name": "Smart Research Assistant",
        "module": 10,
        "date_created": datetime.now().isoformat(),
        "components": {
            "rag_system": "✅ Implementado",
            "web_agent": "✅ Implementado", 
            "memory_manager": "✅ Implementado",
            "intelligent_router": "✅ Implementado",
            "output_parsers": "✅ Implementado",
            "unified_interface": "✅ Implementado"
        },
        "features": [
            "Multi-source information retrieval",
            "Intelligent routing decisions", 
            "Conversation memory management",
            "Structured response formatting",
            "Performance analytics",
            "Modular architecture"
        ],
        "tech_stack": {
            "framework": "LangChain v0.2",
            "llm": "Gemini 2.0 Flash",
            "embeddings": "sentence-transformers/all-MiniLM-L6-v2",
            "vector_store": "FAISS",
            "web_tools": ["DuckDuckGo", "Wikipedia"]
        },
        "next_steps": [
            "Projeto Final 2 (Módulo 11)",
            "Deploy com Streamlit (Módulo 12)",
            "Migração para v1.0 (Módulo 13)"
        ]
    }
    
    return summary

# Salva e mostra o resumo
project_summary = save_project_summary()

print("📄 RESUMO DO PROJETO - MÓDULO 10")
print("="*50)
print(f"🎯 Projeto: {project_summary['project_name']}")
print(f"📅 Data: {project_summary['date_created'][:19]}")
print(f"🤖 LLM: {project_summary['tech_stack']['llm']}")

print("\n🔧 Componentes implementados:")
for comp, status in project_summary['components'].items():
    print(f"  • {comp}: {status}")

print("\n⭐ Features principais:")
for feature in project_summary['features'][:3]:
    print(f"  • {feature}")

print("\n🚀 Próximos passos:")
for step in project_summary['next_steps']:
    print(f"  • {step}")

print("\n🎉 Parabéns! Você completou o Módulo 10!")
print("💪 Agora você tem um assistente de IA completo e funcional!")
print("🔥 Bora para o Projeto Final 2 no próximo módulo!")